<a href="https://colab.research.google.com/github/christianadriano/RL_4_SelfHealingSystems/blob/master/Policy_Evaluation_GridWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

import Failure_Monitor_Environment as env #for future use
env = gym.make('env-v0')

pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv() #initilize environment (gridworld)

def policy_evaluation(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy for a given an environment considering a dicount_factor
    and a stopping factor. Assumes that the environment has a complete
    specification of its dynamics (transition matrix and reward table)
    
    Arguments (inputs):
        policy: [S, A] shaped matrix representing the policy.
 
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
 
        theta: Determines when to stop evaluation once the change in the value 
               function change is less than theta for all states.
 
        discount_factor: Gamma discount factor.
    
    Returns (outputs):
        Vector of length env.nS representing the value function.
    """
    # Initializes state value functions with zero
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backp", i.e., a complete one step ahead
        #update. See Sutton book page 74
        for s in range(env.nS):
            v = 0
            # Look at the available next actions for current state s
            for a, action_prob in enumerate(policy[s]):
                # For each action, update value fucntion by looking
                # at all the reachable next states.
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value. Ref: Sutton book eq. 4.6.
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating if the change in value function is below a theta
        if delta < theta:
            break
    return np.array(V)
